In [ ]:
import numpy as np
import timeit
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook as tqdm

from qrbm.MSQRBM import MSQRBM
from qrbm.classicalRBM import classicalRBM


import matplotlib.pyplot as plt
%matplotlib inline   
plt.rcParams['image.cmap'] = 'gray'

import pandas as pd

from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import img_as_bool

import cv2 as cv
import random
from numpy import genfromtxt

In [ ]:
train_data_df = pd.read_excel('Train.xlsx',header=None)
train_data = np.array(train_data_df)
test_data_df = pd.read_excel('Test.xlsx',header=None)
test_data = np.array(test_data_df)

flat_train_data = []

for i in range(int(len(train_data)/8)):
    x = []
    for k in range(8):
        for j in range(8):
            x.append(train_data[8*i+j][k])
    flat_train_data.append(x)
    
flat_test_data = []

for i in range(int(len(test_data)/8)):
    x = []
    for k in range(8):
        for j in range(8):
            x.append(test_data[8*i+j][k])
    flat_test_data.append(x)

In [ ]:
#Presets

image_height = 8
image_width = 8

len_x = image_height * image_width
len_y = 0

n_visible = 64
n_hidden = 64

epochs = 50
lr = 0.1
lr_decay = 0

In [ ]:
result_picture_tab = []

for i in range(int(len(test_data)/8)):
    x = []
    for k in range(8):
        for j in range(8):
            x.append(test_data[8*i+j][k])
    result_picture_tab.append(x)

In [ ]:
result_picture_tab = []

for i in range(int(len(test_data)/8)):
    x = []
    for k in range(8):
        for j in range(8):
            x.append(test_data[8*i+j][k])
    result_picture_tab.append(x)

bm = MSQRBM(n_visible=n_visible, n_hidden=n_hidden, qpu=True)
bm.image_height = image_height
bm.tqdm = tqdm
bm.result_picture_tab = result_picture_tab

bm.train(flat_train_data, len_x, len_y, epochs = epochs, lr = lr, lr_decay = lr_decay)

weights_biases = bm.get_weights()

In [ ]:
np.savetxt("vishid.csv", weights_biases[0], delimiter=",")
np.savetxt("visbiases.csv", weights_biases[1], delimiter=",")
np.savetxt("hidbiases.csv", weights_biases[2], delimiter=",")

In [ ]:
def compute_metrics(bm):
    rand_label_test = []

    for i in range(len(test_data)):
        rand_mat = np.random.randint(2, size=[2,1])
        #rand_mat = random.choice([0, 1])
        x = np.concatenate((test_data[i][0:6],np.squeeze(rand_mat)),axis = 0).tolist()
        rand_label_test.append(x)

    rand_label_test = np.array(rand_label_test)

    rand_label_picture_tab = []

    for i in range(int(len(rand_label_test)/8)):
        x = []
        for k in range(8):
            for j in range(8):
                x.append(rand_label_test[8*i+j][k])
        rand_label_picture_tab.append(x)
        
    result_picture_tab = []

    for i in range(int(len(test_data)/8)):
        x = []
        for k in range(8):
            for j in range(8):
                x.append(test_data[8*i+j][k])
        result_picture_tab.append(x)

    result_picture_tab = result_picture_tab[0:80]
    rand_label_picture_tab = rand_label_picture_tab[0:80]

    TP = 0
    TN = 0
    FP = 0
    FN = 0

    for i in tqdm(range(len(result_picture_tab))):
        True_image = np.reshape(result_picture_tab[i], (image_width, image_height))
        True_label = True_image[7][0]
        #print('True label: {}'.format(True_label))

        Recon_image = bm.generate(rand_label_picture_tab[i])
        Recon_image = np.reshape(Recon_image, (image_width, image_height))
        Pred_label = int(sum(sum(Recon_image[6:8][:]))/16 > 0.5)
        #print('Predicted label: {}'.format(Pred_label))

        if True_label == Pred_label:
            if True_label == 1:
                TN += 1
            else:
                TP += 1
        else:
            if True_label == 1:
                FP += 1
            else:
                FN += 1

    print('TP (test): {}'.format(TP))
    print('TN (test): {}'.format(TN))
    print('FP (test): {}'.format(FP))
    print('FN (test): {}'.format(FN))

    Accuracy = (TP + TN)/(TP + TN + FP + FN)
    Precision = TP/(TP + FP)
    Recall = TP/(TP + FN)
    F1 = 2*Precision*Recall/(Precision + Recall)

    print('Accuracy (test): {}'.format(Accuracy))
    print('Precision (test): {}'.format(Precision))
    print('Recall (test): {}'.format(Recall))
    print('F1 score (test): {}'.format(F1))

In [ ]:
saved_weights = genfromtxt('vishid_.csv', delimiter=',')
saved_visbiases = genfromtxt('visbiases.csv', delimiter=',')
saved_hidbiases = genfromtxt('hidbiases.csv', delimiter=',')

bm = MSQRBM(n_visible=n_visible, n_hidden=n_hidden, qpu=False)
bm.image_height = image_height
bm.result_picture_tab = result_picture_tab
bm.set_weights(saved_weights,saved_visbiases,saved_hidbiases)
bm.tqdm = tqdm

compute_metrics(bm)